In [4]:
import numpy as np
import librosa
import sounddevice as sd
import tensorflow.keras.models as models

In [5]:

# 모델 로드
MODEL_PATH = "car_detection_cnn_model2_48kHz.h5"
model = models.load_model(MODEL_PATH)

In [8]:


# 실시간 오디오 설정
SAMPLE_RATE = 48000  # 오디오 샘플링 레이트
DURATION = 1  # 분석할 오디오 길이 (초)
CHUNK_SIZE = SAMPLE_RATE * DURATION  # 오디오 데이터의 총 샘플 수

# MFCC 특징 추출 함수
def extract_mfcc(data, sample_rate=48000, max_pad_len=199):
    """
    입력 오디오 데이터로부터 MFCC 특징을 추출합니다.
    :param data: 오디오 신호 데이터
    :param sample_rate: 오디오 샘플링 레이트
    :param max_pad_len: 고정된 입력 길이
    :return: 전처리된 MFCC 데이터
    """
    # MFCC 추출 (40개의 MFCC, 25ms 윈도우, 10.7ms 스텝)
    mfcc = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40, n_fft=2048, hop_length=512)
    # 패딩 또는 자르기
    pad_width = max_pad_len - mfcc.shape[1]
    if pad_width > 0:
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_pad_len]
    return mfcc[np.newaxis, ..., np.newaxis]  # CNN 입력 형식으로 변환

# 실시간 예측 함수
def predict_sound(indata):
    """
    입력 오디오 데이터를 예측하고 결과를 출력합니다.
    :param indata: 실시간 오디오 데이터 (numpy 배열)
    """
    try:
        # 특징 추출
        mfcc_features = extract_mfcc(indata.flatten(), sample_rate=SAMPLE_RATE)
        # 예측
        prediction = model.predict(mfcc_features)
        prediction_value = prediction[0][0]
        print(f"Prediction Value: {prediction_value:.8f}")
        # 결과 출력
        if prediction_value < 0.5:
            print("🚗 Car sound detected!")
        else:
            print("❌ No car sound detected.")
    except Exception as e:
        print(f"Error during prediction: {e}")

# 오디오 스트림 설정
def audio_callback(indata, frames, time, status):
    if status:
        print(status)
    # 오디오 데이터로 예측 수행
    predict_sound(indata)

# 실시간 스트리밍 시작
print("Starting real-time sound detection... Press Ctrl+C to stop.")
try:
    with sd.InputStream(callback=audio_callback, channels=1, samplerate=SAMPLE_RATE, blocksize=CHUNK_SIZE):
        sd.sleep(20000)  # 20초 동안 실행
except KeyboardInterrupt:
    print("Real-time detection stopped.")


Starting real-time sound detection... Press Ctrl+C to stop.
1/1 [==============================] - 0s 23ms/step
Prediction Value: 1.00000000
❌ No car sound detected.
1/1 [==============================] - 0s 23ms/step
Prediction Value: 0.99999988
❌ No car sound detected.
1/1 [==============================] - 0s 19ms/step
Prediction Value: 0.99999982
❌ No car sound detected.
1/1 [==============================] - 0s 22ms/step
Prediction Value: 0.99999982
❌ No car sound detected.
1/1 [==============================] - 0s 22ms/step
Prediction Value: 0.99999905
❌ No car sound detected.
1/1 [==============================] - 0s 23ms/step
Prediction Value: 1.00000000
❌ No car sound detected.
1/1 [==============================] - 0s 21ms/step
Prediction Value: 0.99999666
❌ No car sound detected.
1/1 [==============================] - 0s 21ms/step
Prediction Value: 0.20952760
🚗 Car sound detected!
1/1 [==============================] - 0s 20ms/step
Prediction Value: 0.99986458
❌ No car sound